<H1>Linear Tree Algorithm experiment</H1>

1.) Collect, clean exemplary dataset for regression problem suited for linear tree model.<br>
2.) Develop single tree algorithm with variables for depth, omitted features as input (others to be added later) and feature importance and classification solution as output<br>
3.) Define simple Gradient Boosting algorithm<br>
4.) Bring full algorithm together<br>
5.) Run tests with exemplary dataset<br>

In [18]:
import pandas as pd
import numpy as np
import plotly as pl
import plotly.express as px

pd.set_option('display.max_rows', 500)

In [25]:
#data cleaning and pre-preparation completed in EDA module
df=pd.read_csv('data/ames_housing_price_data_prepared_v01.csv') 
price=df['SalePrice']
price_log = np.log10(price)
#df.head(10).T

<H1>Tree model

In [ ]:
'''
Input: dataframe split into subtrees, current RSS, list of maxdepth per subtree
for each subtree
    if not yet at maxdepth
    for each feature
        for each gap between two observations
            divide tree into two subtrees
            run linar regression on each subtree
            save tuple: (feature, gapposition, RSS-delta)
    find tuple with max RSS-delta, discard remaining tuples
find remaining tuple with max RSS-delta
split dataframe
if all subtrees at maxdepth: return 
else: feed dataframes, RSS current-delta, list of maxdepth per subtree into next tree
'''